In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 맷플롯립 한글 사용 가능
import matplotlib.pyplot as plt
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

# 선택
from scipy.spatial import cKDTree

In [2]:
# 파일 불러오기
# 아파트
apt_df = pd.read_csv('Data_Preprocessing/apt_x_not_distance.csv')
# 공원
park_df = pd.read_csv('../6.Infrastructure/공원좌표.csv', encoding='cp949')
# 버스 정류장
bus_df = pd.read_csv('../4.Public_transportation/Bus_stop_latitude_and_longitude.csv', encoding='cp949')
# 지하철역
station_df = pd.read_csv('../4.Public_transportation/subway_latitude_and_longitude.csv', encoding='cp949')
# 학교
school_df = pd.read_csv('forDP/dp_school.csv')
# 병원
hospital_df = pd.read_csv('forDP/dp_hospital.csv')

In [3]:
# 기본 정보 확인
apt_df.info(), park_df.info(), bus_df.info(), station_df.info(), school_df.info(), hospital_df.info()
apt_df.columns, park_df.columns, bus_df.columns, station_df.columns, school_df.columns, hospital_df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   NO                   251096 non-null  object 
 1   contract_date        251096 non-null  object 
 2   exclusive_area_m2    251096 non-null  float64
 3   floor                251096 non-null  int64  
 4   brand_label_encoded  251096 non-null  int64  
 5   Q_1                  251096 non-null  int64  
 6   Q_2                  251096 non-null  int64  
 7   Q_3                  251096 non-null  int64  
 8   Q_4                  251096 non-null  int64  
 9   apt_age              251096 non-null  int64  
 10  latitude             251096 non-null  float64
 11  longitude            251096 non-null  float64
 12  adjusted_price       251096 non-null  float64
dtypes: float64(4), int64(7), object(2)
memory usage: 24.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries

(Index(['NO', 'contract_date', 'exclusive_area_m2', 'floor',
        'brand_label_encoded', 'Q_1', 'Q_2', 'Q_3', 'Q_4', 'apt_age',
        'latitude', 'longitude', 'adjusted_price'],
       dtype='object'),
 Index(['공원명', 'X좌표(GRS80TM)', 'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)'], dtype='object'),
 Index(['ID', 'bus_stop_name', 'latitude', 'longitude'], dtype='object'),
 Index(['ID', 'name', 'line', 'latitude', 'longitude'], dtype='object'),
 Index(['school_name', 'address', 'latitude', 'longitude'], dtype='object'),
 Index(['hospital_name', 'address', 'latitude', 'longitude'], dtype='object'))

In [4]:
# 결측치 확인
apt_df.isna().sum(), park_df.isna().sum(), bus_df.isna().sum(), station_df.isna().sum(), school_df.isna().sum(), hospital_df.isna().sum()

(NO                     0
 contract_date          0
 exclusive_area_m2      0
 floor                  0
 brand_label_encoded    0
 Q_1                    0
 Q_2                    0
 Q_3                    0
 Q_4                    0
 apt_age                0
 latitude               0
 longitude              0
 adjusted_price         0
 dtype: int64,
 공원명              0
 X좌표(GRS80TM)    11
 Y좌표(GRS80TM)    11
 X좌표(WGS84)      11
 Y좌표(WGS84)      11
 dtype: int64,
 ID               0
 bus_stop_name    0
 latitude         0
 longitude        0
 dtype: int64,
 ID           0
 name         0
 line         0
 latitude     0
 longitude    0
 dtype: int64,
 school_name    0
 address        0
 latitude       0
 longitude      0
 dtype: int64,
 hospital_name    0
 address          0
 latitude         0
 longitude        0
 dtype: int64)

In [5]:
# 여전히 존재하는 미싱밸류 확인 및 처리를 위한 전처리
# mv_school = school_df[school_df.isnull().any(axis=1)]
# mv_hospital = hospital_df[hospital_df.isnull().any(axis=1)]
# mv_school, mv_hospital
# mv_school.to_csv('forDP/mv_school.csv', index=False, encoding='utf-8')
# mv_hospital.to_csv('forDP/mv_hospital.csv', index=False, encoding='utf-8')

In [6]:
# 결측치 처리
apt_df.isna().sum(), park_df.isna().sum(), bus_df.isna().sum(), station_df.isna().sum(), school_df.isna().sum(), hospital_df.isna().sum()
apt_df = apt_df.dropna()
park_df = park_df.dropna()
school_df = school_df.dropna()
hospital_df = hospital_df.dropna()
apt_df.isna().sum(), park_df.isna().sum(), bus_df.isna().sum(), station_df.isna().sum(), school_df.isna().sum(), hospital_df.isna().sum()

(NO                     0
 contract_date          0
 exclusive_area_m2      0
 floor                  0
 brand_label_encoded    0
 Q_1                    0
 Q_2                    0
 Q_3                    0
 Q_4                    0
 apt_age                0
 latitude               0
 longitude              0
 adjusted_price         0
 dtype: int64,
 공원명             0
 X좌표(GRS80TM)    0
 Y좌표(GRS80TM)    0
 X좌표(WGS84)      0
 Y좌표(WGS84)      0
 dtype: int64,
 ID               0
 bus_stop_name    0
 latitude         0
 longitude        0
 dtype: int64,
 ID           0
 name         0
 line         0
 latitude     0
 longitude    0
 dtype: int64,
 school_name    0
 address        0
 latitude       0
 longitude      0
 dtype: int64,
 hospital_name    0
 address          0
 latitude         0
 longitude        0
 dtype: int64)

In [7]:
# 컬럼 정리
apt_df = apt_df[['NO', 'contract_date', 'latitude', 'longitude']]

park_df.columns = ['park_name', 'x', 'y', 'longitude', 'latitude']
park_df = park_df[['park_name', 'latitude', 'longitude']]

bus_df.columns = ['bus_id', 'bus_stop_name', 'latitude', 'longitude']

station_df['line_name'] = station_df['line'] + '_' + station_df['name']
station_df = station_df.drop_duplicates(subset='line_name')
station_df.columns = ['station_id', 'station_name', 'line', 'latitude', 'longitude', 'line_name']
station_df = station_df[['station_id', 'line_name', 'latitude', 'longitude']]

school_df = school_df[['school_name', 'latitude', 'longitude']]

hospital_df = hospital_df[['hospital_name', 'latitude', 'longitude']]

In [8]:
apt_df.describe(), station_df.describe(), bus_df.describe(), park_df.describe(), school_df.describe(), hospital_df.describe()

(            latitude      longitude
 count  251096.000000  251096.000000
 mean       37.555110     126.995693
 std         0.056496       0.090399
 min        37.434420     126.806400
 25%        37.506884     126.917908
 50%        37.549193     127.017588
 75%        37.599466     127.065709
 max        37.687503     127.179998,
         station_id    latitude   longitude
 count   782.000000  782.000000  782.000000
 mean   2389.792839   37.511847  126.971378
 std    1593.851659    0.154688    0.182881
 min     150.000000   36.769502  126.441442
 25%    1285.250000   37.470522  126.853044
 50%    2511.500000   37.521754  126.997706
 75%    3137.750000   37.577946  127.073403
 max    9996.000000   38.100730  127.723792,
              bus_id      latitude     longitude
 count  1.129000e+04  11290.000000  11290.000000
 mean   1.132294e+08     37.550071    126.985975
 std    6.993850e+06      0.054553      0.086257
 min    1.000000e+08     37.430537    126.721031
 25%    1.079001e+08    

In [9]:
def calculate_nearest_facilities_with_scores(apt_df, ref_df, ref_name, radius_km_list=[0.5, 1, 1.5], 
                                             scores=[10, 5, 1], name_column='name'):
    """
    아파트 기준으로 가장 가까운 시설 이름 및 거리 계산 (중복 방지) + 점수 계산 추가.
    apt_df: 아파트 데이터프레임
    ref_df: 기준 데이터프레임 (공원, 버스정류장, 지하철역 등)
    ref_name: 기준 이름 ('공원', '버스정류장', '지하철역' 등)
    radius_km_list: 계산할 반경 (km 단위)
    scores: 반경별 가중치 점수 (반경 순서에 맞게 입력)
    name_column: 시설 이름 컬럼 (기본값 'name')
    """
    # 결과 저장용 딕셔너리
    results = {f'{ref_name}_within_{r}km': [] for r in radius_km_list}
    results[f'closest_{ref_name}_dist_km'] = []
    results[f'closest_{ref_name}_name'] = []
    total_score_col = f'{ref_name}_score'  # 점수 컬럼 이름
    results[total_score_col] = []

    # KDTree 생성 (ref_df 기준 좌표를 라디안으로 변환)
    ref_tree = cKDTree(np.radians(ref_df[['latitude', 'longitude']].values))

    for _, apt_row in apt_df.iterrows():
        apt_coords = np.radians([apt_row['latitude'], apt_row['longitude']])  # 아파트 좌표 라디안 변환
        seen_indices = set()  # 중복 방지용 인덱스 저장
        total_score = 0  # 점수 초기화

        for radius_km, score in zip(radius_km_list, scores):
            radius_rad = radius_km / 6371.0  # 반경을 라디안 단위로 변환
            indices = ref_tree.query_ball_point(apt_coords, r=radius_rad)

            # 반경 내 고유 시설 개수 계산
            unique_indices = [idx for idx in indices if idx not in seen_indices]
            seen_indices.update(unique_indices)  # 이미 처리한 인덱스 추가

            count = len(unique_indices)
            results[f'{ref_name}_within_{radius_km}km'].append(count)

            # 점수 계산 (반경 내 개수 * 가중치)
            total_score += count * score

        # 가장 가까운 거리 및 인덱스 계산
        dist, idx = ref_tree.query(apt_coords, k=1)
        closest_distance = dist * 6371.0  # 라디안 -> km 변환
        closest_name = ref_df.iloc[idx][name_column]  # 인덱스를 통해 시설 이름 가져오기

        results[f'closest_{ref_name}_dist_km'].append(closest_distance)
        results[f'closest_{ref_name}_name'].append(closest_name)
        results[total_score_col].append(total_score)  # 최종 점수 저장

    # 결과를 apt_df에 추가
    for col_name, col_values in results.items():
        apt_df[col_name] = col_values

    return apt_df


In [10]:
# 지하철역 (반경 500m, 1km, 1.5km -> 점수 10, 5, 1)
apt_df = calculate_nearest_facilities_with_scores(apt_df, station_df, 'station', 
                                                  radius_km_list=[0.5, 1, 1.5], scores=[10, 5, 1], 
                                                  name_column='line_name')
# 버스정류장 (반경 500m, 1km, 1.5km -> 점수 10, 5, 1)
apt_df = calculate_nearest_facilities_with_scores(apt_df, bus_df, 'bus_stop', 
                                                  radius_km_list=[0.5, 1, 1.5], scores=[10, 5, 1], 
                                                  name_column='bus_stop_name')
# 공원 (반경 500m, 1km, 1.5km -> 점수 10, 5, 1)
apt_df = calculate_nearest_facilities_with_scores(apt_df, park_df, 'park', 
                                                  radius_km_list=[0.5, 1, 1.5], scores=[10, 5, 1], 
                                                  name_column='park_name')
# 병원
apt_df = calculate_nearest_facilities_with_scores(apt_df, hospital_df, 'hospital', 
                                                  radius_km_list=[0.5, 1, 1.5], scores=[10, 5, 1], 
                                                  name_column='hospital_name')
# 학교
apt_df = calculate_nearest_facilities_with_scores(apt_df, school_df, 'school', 
                                                  radius_km_list=[0.5, 1, 1.5], scores=[10, 5, 1], 
                                                  name_column='school_name')

In [11]:
# 새로 추가된 데이터를 확인
apt_df.info()
print(apt_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251096 entries, 0 to 251095
Data columns (total 34 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   NO                        251096 non-null  object 
 1   contract_date             251096 non-null  object 
 2   latitude                  251096 non-null  float64
 3   longitude                 251096 non-null  float64
 4   station_within_0.5km      251096 non-null  int64  
 5   station_within_1km        251096 non-null  int64  
 6   station_within_1.5km      251096 non-null  int64  
 7   closest_station_dist_km   251096 non-null  float64
 8   closest_station_name      251096 non-null  object 
 9   station_score             251096 non-null  int64  
 10  bus_stop_within_0.5km     251096 non-null  int64  
 11  bus_stop_within_1km       251096 non-null  int64  
 12  bus_stop_within_1.5km     251096 non-null  int64  
 13  closest_bus_stop_dist_km  251096 non-null  f

In [12]:
apt_df['total_score'] = apt_df['station_score'] + apt_df['bus_stop_score'] + apt_df['park_score'] \
                    + apt_df['hospital_score'] + apt_df['school_score']

In [13]:
apt_df.columns

Index(['NO', 'contract_date', 'latitude', 'longitude', 'station_within_0.5km',
       'station_within_1km', 'station_within_1.5km', 'closest_station_dist_km',
       'closest_station_name', 'station_score', 'bus_stop_within_0.5km',
       'bus_stop_within_1km', 'bus_stop_within_1.5km',
       'closest_bus_stop_dist_km', 'closest_bus_stop_name', 'bus_stop_score',
       'park_within_0.5km', 'park_within_1km', 'park_within_1.5km',
       'closest_park_dist_km', 'closest_park_name', 'park_score',
       'hospital_within_0.5km', 'hospital_within_1km', 'hospital_within_1.5km',
       'closest_hospital_dist_km', 'closest_hospital_name', 'hospital_score',
       'school_within_0.5km', 'school_within_1km', 'school_within_1.5km',
       'closest_school_dist_km', 'closest_school_name', 'school_score',
       'total_score'],
      dtype='object')

In [15]:
selected_apt_df =apt_df[['NO', 'contract_date', 
                         'station_within_0.5km', 'station_within_1km', 
                         'station_within_1.5km', 'closest_station_dist_km',
                         'bus_stop_within_0.5km', 'bus_stop_within_1km', 
                         'bus_stop_within_1.5km', 'closest_bus_stop_dist_km',
                         'park_within_0.5km', 'park_within_1km', 
                         'park_within_1.5km', 'closest_park_dist_km',
                         'hospital_within_0.5km', 'hospital_within_1km', 
                         'hospital_within_1.5km', 'closest_hospital_dist_km',
                         'school_within_0.5km', 'school_within_1km', 
                         'school_within_1.5km', 'closest_school_dist_km',
                         'station_score', 'bus_stop_score', 'park_score', 
                         'hospital_score', 'school_score', 'total_score',
                         'latitude', 'longitude']]
# 사용하지 않을 컬럼: (거리 계산 잘 되었는지 확인만!)
# 'closest_station_name', 'closest_bus_stop_name', 'closest_park_name',
# 'closest_hospital_name', 'closest_school_name'
# 사용할 컬럼
       # 인덱스용 : 'NO', 'contract_date'
       # 거리(중복 집계 제거)
              # 'station_within_0.5km', 'station_within_1km', 
              # 'station_within_1.5km', 'closest_station_dist_km',
              # 'bus_stop_within_0.5km', 'bus_stop_within_1km', 
              # 'bus_stop_within_1.5km', 'closest_bus_stop_dist_km',
              # 'park_within_0.5km', 'park_within_1km', 
              # 'park_within_1.5km', 'closest_park_dist_km',
              # 'hospital_within_0.5km', 'hospital_within_1km', 
              # 'hospital_within_1.5km', 'closest_hospital_dist_km',
              # 'school_within_0.5km', 'school_within_1km', 
              # 'school_within_1.5km', 'closest_school_dist_km',
              # 'station_score', 'bus_stop_score', 'park_score', 
              # 'hospital_score', 'school_score', 'total_score'
              # 'latitude', 'longitude'

In [17]:
selected_apt_df.head()

,NO,contract_date,station_within_0.5km,station_within_1km,station_within_1.5km,closest_station_dist_km,bus_stop_within_0.5km,bus_stop_within_1km,bus_stop_within_1.5km,closest_bus_stop_dist_km,...,school_within_1.5km,closest_school_dist_km,station_score,bus_stop_score,park_score,hospital_score,school_score,total_score,latitude,longitude
0,2019_00001,2019-12-31,1,4,7,0.424042,21,40,99,0.177637,...,5,0.143573,37,509,15,682,45,1288,37.569628,127.041524
1,2019_00002,2019-12-31,0,1,1,0.744370,13,49,70,0.139836,...,5,0.336639,6,445,1,514,65,1031,37.589283,127.072500
2,2019_00003,2019-12-31,0,0,2,1.202505,15,29,80,0.214917,...,6,0.233091,2,375,16,977,71,1441,37.574609,127.060129
3,2019_00004,2019-12-31,0,1,1,0.878111,11,38,36,0.147960,...,5,0.536320,6,336,5,119,30,496,37.616686,127.001842
4,2019_00005,2019-12-31,0,0,1,1.384230,18,47,78,0.208754,...,12,0.200016,1,493,2,785,42,1323,37.576338,127.074573


In [19]:
selected_apt_df.to_csv('Data_Preprocessing/apt_with_distance.csv', index = False, encoding='utf-8')